In [ ]:
# read in the credentials as environment variables
import sys
import os
cwd = os.getcwd()
if cwd.__contains__('src'):
    env_vars = !cat /src/.env
elif cwd.startswith('/data/users/jqhu'):
    env_vars = !cat /data/users/jqhu/work/tamagotchi/smartFlies/.env
    sys.path.append('/data/users/jqhu/work/tamagotchi/smartFlies')
else:
    env_vars = !cat /gscratch/walkerlab/jqhu/smartFlies/tamagotchi/.env

for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

from schemas.schema_v4 import *

In [ ]:
# (TrainingConfig() & f'training_config_hash="074eef10d15f46153fe6433861b160b8"')
for f in schema.jobs.fetch():
    error = f[4].split(':')[0:2]
    if error[0] == 'ConnectionResetError':
        print(f[1]) # print hash
        
# (schema.jobs & 'key_hash="c89f5ef984bfb3e30b3db6ff6ae68fa6"').delete()

In [ ]:
key_to_train = {}
for entry in schema.jobs.fetch():
    key_hash = entry[1]
    status = entry[4]
    train_hash = entry[3]['training_config_hash']
    key_to_train[key_hash] = [train_hash, status]
    
# find in the training config table
for key_hash, tpl in key_to_train.items():
    train_hash = tpl[0]
    status = tpl[1]
    training_config = TrainingConfig()
    print(f"key_hash: {key_hash}, train_hash: {train_hash}")
    training_config = (training_config & f'training_config_hash="{train_hash}"')
    
    print(training_config.fetch('save_dir'), "seed:", training_config.fetch('seed'), status)


In [ ]:
schema.jobs


        
    
# print((TrainingConfig() & f'training_config_hash="e169334e293b3b147111af579743d43a"').fetch()[['save_dir', 'seed']])
# (schema.jobs & 'host="g3112"').fetch() # CNC DEV MACHINE

# (schema.jobs & 'key_hash="ecb15d226575cf3c8716c339bd1eeb30"').delete()
# (schema.jobs & 'status="error"' & 'key_hash="0bb9a16627e73f4077ef6a5c2fbc67b9"').delete()
# (schema.jobs & 'key_hash="93935d32595f366ec6d0adf707e801be"').delete()

# schema.jobs.delete()
# training_config_hash_to_delete = ["8171898c23c5a659f66124c1e971b1b4", "7d7cfda61322daa8d05d557dbc8fa2c7", "8a14ed9755f248b49c29679f7ac785fa"]
# corresponding_ket_hash = ["001e39a0d09286e887e89d0c93f3e1da", "5ed4bd5907d1356fa70a56fe5c1e0f91"]


###### find the training config hash and the seed to insert into the training result table  ########

# result_to_insert = {}
# for i in (schema.jobs & 'status="error"').fetch():
#     conf = TrainingConfig() & f'training_config_hash="{i[3]["training_config_hash"]}"'
#     if conf:
#         print(i[1], i[3]['training_config_hash'], conf.fetch('seed')[0], conf.fetch('save_dir'))
#         result_to_insert[i[1]] = [i[3]['training_config_hash'], conf.fetch('seed')[0]]
#     else:
#         print(i[1], i[3]['training_config_hash'], "not found in training config")

# corresponding_ket_hash = result_to_insert.keys()
# for h in corresponding_ket_hash:
#     print((schema.jobs & f'key_hash="{h}"'))
#     (schema.jobs & f'key_hash="{h}"').delete()

# for k in result_to_insert.keys():
#     print(k, result_to_insert[k][0], result_to_insert[k][1])
#     TrainingResult().insert(
#         [dict(training_config_hash=result_to_insert[k][0],
#             seed=result_to_insert[k][1],
#             hours_elapsed=0)]
#         , allow_direct_insert=True
#     )
# TrainingResult().insert(
    
#     [dict(training_config_hash='e169334e293b3b147111af579743d43a',
#         seed=22720,
#         hours_elapsed=0)]
#     , allow_direct_insert=True
# )


In [ ]:
TrainingConfig()
# (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/gnODE_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/"' & 'rnn_type="GNODE"')
# (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/"')
# (TrainingConfig() & 'seed="1477"')
# (TrainingConfig() & 'loc_algo="quantile"')
# (TrainingConfig() & 'env_dt="0.1"')
# (TrainingConfig() & 'rnn_type="GNODE"')

# (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind"')

# TrainingConfig().fetch('save_dir')


# for hash in ['c97448325b629989b2d82e8f2f325c55', '45513dd8ac9d9cdbb3a34f957436f7af', 'dee781bba1737b60f0e9b494f6acfdfc']:
#     print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()[['save_dir', 'seed']])
# (TrainingConfig() & 'training_config_hash="d1cc1b54c140f764025f8fc169a34456"').fetch()[['save_dir', 'seed']]

In [ ]:
# (TrainingConfig() & 'training_config_hash="8a14ed9755f248b49c29679f7ac785fa"')

In [ ]:
# TrainingResult.populate(reserve_jobs=True)
# 8171898c23c5a659f66124c1e971b1b4
# 7d7cfda61322daa8d05d557dbc8fa2c7
# 8a14ed9755f248b49c29679f7ac785fa
# 
# TrainingResult().insert(
    
#     [dict(training_config_hash='8a14ed9755f248b49c29679f7ac785fa',
#         seed=15717,
#         hours_elapsed=0)]
#     , allow_direct_insert=True
# )

# TrainingResult() & 'training_config_hash="8a14ed9755f248b49c29679f7ac785fa"'
TrainingResult()
TrainingResult() & 'seed="6409"'

In [ ]:
training_config_hash = [item[0] for item in TrainingResult().fetch()]
training_config = [item[0] for item in TrainingConfig().fetch()]
to_be_done_exp = set(training_config).difference(set(training_config_hash))
print(len(to_be_done_exp))
undone = ['/plume_10057_5e0b837fa56b6a4dffff1a034adcff23.pt',
'/plume_11357_85f2b252e3fb3537cc8b97e7299543d1.pt',
'/plume_13356_edf288d268883f38671e8d8b3c6faa80.pt',
'/plume_19417_4ae9997221924be5c7af5b45ebc2f399.pt',
'/plume_23197_87f2d20ceaa4d4567390aef42c9a18b3.pt',
'/plume_23311_010d39e8723185121880b9cb06092819.pt',
'/plume_31287_f381b0d5c73c6a567dc6a53437118d1b.pt',
'/plume_6337_48886f495fca96cc3bd4e5fae0eff333.pt',
"/plume_8898_d8e5f14b5b73bcc1d45585b57d7f6b8c.pt"]

undone = [exp.split('_')[-1].replace('.pt', '') for exp in undone]
print(len(undone))
print([exp for exp in to_be_done_exp if exp in undone])
to_be_done_exp
for hash in to_be_done_exp:
    print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()['seed'])
# (TrainingConfig() & 'training_config_hash="061f81f6e646a49a94eb2d27c0bf41ef"')

# insert a hyak-based example 061824

In [ ]:
dict_to_insert = {
 'save_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/gnODE_loc_algo_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/',
#  'outsuffix': '', # to be built by insert 
 'num_processes': 4, # nproc=8 for 4 concurrently agents. 
#  'seed': 2894, # to be sampled when inserting
 'dataset': ['constantx5b5', 'constant_mag_narrowx5b5', 'poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5'],
 'num_env_steps': 10000000,
 'birthx': 0.01, # TODO: verify this is what Toha has
 'qvar': [2.0, 1.0, 0.5, 0.5], # Variance of init. location; higher = more off-plume initializations. Note this is set to 0 in evalCli 
 'diff_max': [0.8, 0.8, 0.8, 0.8],
 'diff_min': [0.7, 0.7, 0.4, 0.4],
 'apparent_wind': True,
 'apparent_wind_allo': False,
 'visual_feedback': True,
 'birthx_linear_tc_steps': 7, 
 'birthx_max': 1.0,
 'rnn_type': 'GNODE',
 'hidden_size': 16,
 'env_dt': 0.04,
 'dryrun': False,
 'algo': 'ppo',
 'lr': 0.0003,
 'eps': 1e-05,
 'alpha': 0.99,
 'gamma': 0.99,
 'use_gae': True,
 'gae_lambda': 0.95,
 'entropy_coef': 0.005,
 'value_loss_coef': 0.5,
 'max_grad_norm': 0.5,
 'cuda_deterministic': False,
 'num_steps': 2048,
 'ppo_epoch': 10,
 'num_mini_batch': 4,
 'clip_param': 0.2,
 'log_interval': 1,
 'save_interval': 100,
 'no_cuda': False,
 'use_proper_time_limits': False,
 'recurrent_policy': True,
 'use_linear_lr_decay': True,
 'env_name': 'plume',
 'log_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/logs/',
 'dynamic': False,
 'eval_type': 'skip',
 'eval_episodes': 20,
 'eval_interval': None,
 'weight_decay': 0.0001,
 'betadist': False,
 'stacking': 0,
 'masking': None,
 'stride': 1,
 'curriculum': False,
 'turnx': 1.0,
 'movex': 1.0,
 'auto_movex': False,
 'auto_reward': False,
 'loc_algo': 'quantile',
 'time_algo': 'uniform',
 'walking': False,
 'radiusx': 1.0,
 'diffusion_min': 1.0,
 'diffusion_max': 1.0,
 'r_shaping': ['step', 'oob'],
 'wind_rel': True,
 'action_feedback': False,
 'squash_action': True,
 'flipping': True,
 'odor_scaling': True,
 'stray_max': 2.0,
 'test_episodes': 50,
 'viz_episodes': 10,
 'model_fname': '',
 'obs_noise': 0.0, # 16deg = 0.28 rad
 'act_noise': 0.0,
 'cuda': True,
 'if_vec_norm': 1,
 'if_train_actor_std': True,
 }

In [ ]:
for i in range(15):
    TrainingConfig.insert1(dict_to_insert)

In [ ]:
TrainingConfig().fetch()['outsuffix']
len(TrainingConfig().fetch()['outsuffix'])


In [ ]:
schema.jobs

# filtering table entries

In [ ]:
TrainingConfig & 'lr < 1'
TrainingConfig().fetch()['save_dir']

# scaping the table

In [ ]:
TrainingConfig.drop()